In [2]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [3]:
!pip install pandas pyarrow
!pip install pandas fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np

# خواندن داده‌ها از فایل CSV
input_file = '//content//mydrive/MyDrive//Colab Notebooks//formation_damage_optimized.parquet'  # نام فایل ورودی
output_cleaned = '//content//mydrive/MyDrive//Colab Notebooks//cleaned_data.csv'
output_removed = '//content//mydrive/MyDrive//Colab Notebooks//removed_outliers.csv'

df = pd.read_parquet(input_file)

# ستون‌های مورد نظر برای آنالیز
columns_to_check = ['Temperature_C', 'Pressure_psi', 'Permeability_mD', 'Flow_Rate_bbl_day']

# تعریف تابع تشخیص پرت با روش IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# ذخیره داده‌های غیرمعمول
removed_outliers = pd.DataFrame()

# پردازش هر ستون
for col in columns_to_check:
    if col not in df.columns:
        print(f"⚠️ ستون '{col}' وجود ندارد و از پردازش حذف شد.")
        continue

    outliers, lb, ub = detect_outliers_iqr(df, col)
    print(f"\n📊 {col}:")
    print(f"حد پایین IQR: {lb:.2f}, حد بالای IQR: {ub:.2f}")
    print(f"تعداد مقادیر پرت: {len(outliers)}")

    removed_outliers = pd.concat([removed_outliers, outliers], axis=0)

# حذف سطرهای دارای مقادیر پرت از داده‌های اصلی
cleaned_df = df.drop(removed_outliers.index).reset_index(drop=True)

# ذخیره داده‌های تمیز شده
cleaned_df.to_csv(output_cleaned, index=False)
print(f"\n✅ داده‌های تمیز شده در فایل '{output_cleaned}' ذخیره شد.")

# ذخیره مقادیر پرت در یک فایل جداگانه
if not removed_outliers.empty:
    removed_outliers.to_csv(output_removed, index=False)
    print(f" مقادیر پرت در فایل '{output_removed}' ذخیره شد.")
else:
    print(" هیچ مقدار پرتی یافت نشد.")


📊 Temperature_C:
حد پایین IQR: -28.76, حد بالای IQR: 286.24
تعداد مقادیر پرت: 315609

📊 Pressure_psi:
حد پایین IQR: -6349.49, حد بالای IQR: 23045.47
تعداد مقادیر پرت: 248105

📊 Permeability_mD:
حد پایین IQR: -262.60, حد بالای IQR: 787.72
تعداد مقادیر پرت: 478607

📊 Flow_Rate_bbl_day:
حد پایین IQR: -772.03, حد بالای IQR: 2357.10
تعداد مقادیر پرت: 455165

✅ داده‌های تمیز شده در فایل '//content//mydrive/MyDrive//Colab Notebooks//cleaned_data.csv' ذخیره شد.
 مقادیر پرت در فایل '//content//mydrive/MyDrive//Colab Notebooks//removed_outliers.csv' ذخیره شد.
